In [1]:
'''
with open('file.txt','r') as f:
    lines = f.readlines()
    for i in range(len(lines)):
        if lines[i][-1] == '\n':
            lines[i] = lines[i][:-1]
'''

with open('file.txt','r') as f:
    lines = f.readlines()
    lines = [line.replace('\n','') for line in lines]

lines = list(filter(lambda x: x!='', lines))    
print(lines)

['a,c,d,e,g,h,k', 'k+l+m=>i', 'i+l+j=>q', 'c+d+e=>b', 'a+b=>q', 'l+n+o+p=>q', 'c+h=>r', 'r+j+m=>s', 'f+h=>b', 'g=>f', 'q']


In [2]:
knowledge_base = set([kn for kn in lines[0].split(',')])
print(knowledge_base)

{'h', 'd', 'g', 'a', 'e', 'c', 'k'}


In [3]:
rule_base_rev = {tuple(rule_line.split('=>')[0].split('+')):rule_line.split('=>')[1] for rule_line in lines[1:-1]}
rule_base = {val:[] for val in rule_base_rev.values()}
for key,val in rule_base_rev.items():
    rule_base[val].append(key)
print(rule_base_rev)
print(rule_base)

{('k', 'l', 'm'): 'i', ('i', 'l', 'j'): 'q', ('c', 'd', 'e'): 'b', ('a', 'b'): 'q', ('l', 'n', 'o', 'p'): 'q', ('c', 'h'): 'r', ('r', 'j', 'm'): 's', ('f', 'h'): 'b', ('g',): 'f'}
{'i': [('k', 'l', 'm')], 'q': [('i', 'l', 'j'), ('a', 'b'), ('l', 'n', 'o', 'p')], 'b': [('c', 'd', 'e'), ('f', 'h')], 'r': [('c', 'h')], 's': [('r', 'j', 'm')], 'f': [('g',)]}


In [4]:
goal_var = lines[-1]
print(goal_var)

q


In [5]:
'''
# Forward Chaining
def forward_chaining(knowledge_base, rule_base):
    new_knowledge_base = knowledge_base.copy()
    print(new_knowledge_base)
    prev_len = len(new_knowledge_base)
    while True:
        for premise,conclusion in rule_base.items():
            if all([premise_k in new_knowledge_base for premise_k in premise]) :
                new_knowledge_base.update(conclusion)
        if prev_len == len(new_knowledge_base):
            break
        prev_len = len(new_knowledge_base)
    print(new_knowledge_base)

forward_chaining(knowledge_base, rule_base)
'''
pass

In [6]:
'''
# Backward chaining
def backward_chaining(knowledge_base, rule_base, goal_var):
    if goal_var in knowledge_base:
        return True
    for premise,conclusion in rule_base.items():
        if conclusion==goal_var:
            premise_truth_vals = []
            if goal_var == initial_goal_state:
                paths.append([])
            for var in premise:
                premise_truth_vals.append(backward_chaining(knowledge_base, rule_base, var))
            if all(premise_truth_vals):
                knowledge_base.update(goal_var)
                paths[-1].append((premise, conclusion))
                return True
    return False

paths = []
initial_goal_state = goal_var
print(backward_chaining(knowledge_base.copy(), rule_base, goal_var), paths[-1])
'''
pass

In [13]:
# Forward Chaining
def forward_chaining(knowledge_base, rule_base):
    new_knowledge_base = knowledge_base.copy()
    prev_len = len(new_knowledge_base)
    while True:
        for conclusion,premises_list in rule_base.items():
            for premises in premises_list:
                if all([premise in new_knowledge_base for premise in premises]) :
                    new_knowledge_base.update(conclusion)
        if prev_len == len(new_knowledge_base):
            break
        prev_len = len(new_knowledge_base)
    return new_knowledge_base

print('Initial Knowledge:', knowledge_base)
print('Final Knowledge:', forward_chaining(knowledge_base, rule_base))
print('Knowledge Added:', forward_chaining(knowledge_base, rule_base) - knowledge_base)

Initial Knowledge: {'h', 'd', 'g', 'a', 'e', 'c', 'k'}
Final Knowledge: {'h', 'q', 'f', 'd', 'g', 'a', 'e', 'b', 'r', 'c', 'k'}
Knowledge Added: {'f', 'b', 'r', 'q'}


In [10]:
# Backward chaining
def backward_chaining(knowledge_base, rule_base, goal_var):
    # Globals to trace path
    global paths
    global initial_goal_state
    
    if goal_var in knowledge_base:
        if len(paths)==0:
            paths.append([goal_var])
        else:
            paths[-1].append(goal_var)
        return True
    
    if goal_var in rule_base:
        for premises in rule_base[goal_var]:
            if goal_var==initial_goal_state:
                paths.append([])
            all_satisfied=True
            for premise in premises:
                if not backward_chaining(knowledge_base, rule_base, premise):
                    all_satisfied=False
                    break
            if all_satisfied:
                knowledge_base.update(goal_var)
                paths[-1].append(str(premises)+'=>'+goal_var)
                return True
                
    return False

paths = []
initial_goal_state = goal_var
res = backward_chaining(knowledge_base.copy(), rule_base, goal_var)
if res:
    print('Proved Path:', paths[-1])

Proved Path: ['a', 'c', 'd', 'e', "('c', 'd', 'e')=>b", "('a', 'b')=>q"]
